In [ ]:
# run after processing AE calibration

# --- Set up directories ---

import os

# === Define input and output folders ===
folder_balldrop = r'C:\...\balldrop' #update the path to 'balldrop' folder
folder_AE = r'C:\...\AE' #update the path to 'AE' folder
folder_processed = r'C:\...\output' #update the path to 'output' folder

# ======================================

# Verify that input folders exist
for folder in [folder_balldrop, folder_AE]:
    if not os.path.exists(folder):
        raise FileNotFoundError(f"Input folder not found: {folder}")

# Create the output folder if it doesn't exist
os.makedirs(folder_processed, exist_ok=True)

# Confirm setup
print("Folder set-up completed.")


In [ ]:
# --- SNR estimation ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal.windows import blackmanharris
import os

# === Load waveform data ===

waveform_file = os.path.join(folder_processed, "AE_experiment.csv")
waveform_df = pd.read_csv(waveform_file)

# ===========================

# Check required columns
required_columns = ['waveform_num', 'time_s', 'amplitude']
if not all(col in waveform_df.columns for col in required_columns):
    raise ValueError(f"Missing required columns: {required_columns}")

# Compute SNR for each waveform
snr_results = []

for waveform_num, group in waveform_df.groupby("waveform_num"):
    time = group["time_s"].values
    amplitude = np.nan_to_num(group["amplitude"].values)

    # Apply Blackman-Harris window
    window = blackmanharris(len(amplitude))
    amplitude_windowed = amplitude * window

    # FFT computation
    dt = time[1] - time[0]  # time step
    freqs = np.fft.rfftfreq(len(time), d=dt)
    fft_result = np.fft.rfft(amplitude_windowed)
    amplitude_spectrum = np.abs(fft_result) / len(time)

    # Define noise band and signal band
    noise_band = freqs < 0.5e-5
    signal_band = ~noise_band

    # Compute power
    power_signal = np.sum(amplitude_spectrum[signal_band]**2)
    power_noise = np.sum(amplitude_spectrum[noise_band]**2)

    # Calculate spectral SNR in dB
    if power_noise > 0:
        snr_db = 10 * np.log10(power_signal / power_noise)
    else:
        snr_db = np.nan

    snr_results.append({"waveform_num": waveform_num, "snr_spec": snr_db})

# Assemble results into DataFrame
snr_df = pd.DataFrame(snr_results).dropna()

# Normalize SNR (min-max scaling)
snr_min = snr_df["snr_spec"].min()
snr_max = snr_df["snr_spec"].max()
snr_df["snr_spec_norm"] = (snr_df["snr_spec"] - snr_min) / (snr_max - snr_min)

# Save results
snr_output_file = os.path.join(folder_processed, "SNR_results.csv")
snr_df.to_csv(snr_output_file, index=False)
print(f"SNR results saved to: {snr_output_file}")

# Preview results 
print("\nPreview of SNR results:")
print(snr_df.head())

# Plot histogram of SNR 
plt.figure(figsize=(7, 4))
plt.hist(snr_df["snr_spec"], bins=50, color="blue", alpha=0.7, edgecolor="black")
plt.xlabel("SNR", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.title("Histogram of SNR", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()
